In [ ]:
! pip install textattack  sentence_transformers evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 418 kB 30.5 MB/s 
     |████████████████████████████████| 85 kB 2.6 MB/s 
     |████████████████████████████████| 72 kB 641 kB/s 
     |████████████████████████████████| 60 kB 2.5 MB/s 
     |████████████████████████████████| 125 kB 61.8 MB/s 
     |████████████████████████████████| 769 kB 63.8 MB/s 
     |████████████████████████████████| 3.6 MB 62.5 MB/s 
     |████████████████████████████████| 5.8 MB 80.0 MB/s 
     |████████████████████████████████| 401 kB 98.4 MB/s 
     |████████████████████████████████| 41.4 MB 1.3 MB/s 
     |████████████████████████████████| 365 kB 80.2 MB/s 
     |████████████████████████████████| 182 kB 88.7 MB/s 
     |████████████████████████████████| 95 kB 5.6 MB/s 
     |████████████████████████████████| 132 kB 83.7 MB/s 
     |████████████████████████████████| 212 kB 104.2 MB/s 
     |████████████████████████████████

In [ ]:
import textattack
import pandas as pd
import json

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from datasets import load_dataset, load_from_disk, concatenate_datasets

In [ ]:
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset
from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker
from textattack import AttackArgs
from textattack.datasets import Dataset

# Import the model
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset

In [ ]:
root = "/content/drive/Shareddrives/BipinA/EE8227_secure_ml/Project"

In [ ]:
from datasets import load_from_disk

In [ ]:
dataset = load_from_disk(f"{root}/data/imdb_test_6k.hf")

In [ ]:
_dataset = dataset.shuffle(seed=42).select(range(1000))

In [ ]:
_dataset.to_pandas()

,text,label
0,"If I could give this movie less than a 1, I wo...",0
1,I mistakenly thought this was the 70's art fil...,0
2,The original story and funny compelling charac...,1
3,"First off, I'm a huge Bronson fan, have been s...",0
4,It's dreadful rubbish. I liked 'How Do You Wan...,0
...,...,...
995,A film of high intelligence and activity. As w...,1
996,"From the late teens to the 1920s, Stan Laurel ...",0
997,I saw this back in 99 and I remember loving it...,1
998,"""Labyrinth of Passion""--with the possible exce...",1


In [ ]:
test_dataset = HuggingFaceDataset(_dataset.select(range(0,500)))

In [ ]:
len(test_dataset)

500

In [ ]:
checkpoint_name = "textattack/bert-base-uncased-imdb"

model = transformers.AutoModelForSequenceClassification.from_pretrained(checkpoint_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(checkpoint_name)

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Create the goal function using the model
goal_function = UntargetedClassification(model_wrapper)


Downloading:   0%|          | 0.00/511 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
import torch

# Attack 1: BAEGarg2019

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from textattack.attack_recipes import BAEGarg2019, BERTAttackLi2020

In [ ]:
BAEGarg2019

textattack.attack_recipes.bae_garg_2019.BAEGarg2019

In [ ]:
attack = BERTAttackLi2020.build(model_wrapper)
attack

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bert-attack
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  48
    (min_confidence):  0.0005
  )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_percent):  0.4
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.2
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
)

In [ ]:
attack = BAEGarg2019.build(model_wrapper)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
import numpy as np

In [ ]:
import torch

In [ ]:
import os 

In [ ]:
torch.cuda.device_count()

1

In [ ]:
attack_args = AttackArgs(num_examples=-1,
                        parallel = bool(torch.cuda.device_count()),
                        log_to_csv=f"{root}/data/log_BAEGarg2019_attack_BERT_1.csv",
                        checkpoint_interval=100, 
                        checkpoint_dir=f"{root}/checkpoints_log_BAEGarg2019_attack_BERT_1",
                        csv_coloring_style = 'plain',
                        disable_stdout=True,
                        shuffle=True,
                        enable_advance_metrics=True,
                        silent=True
                        )

attacker = Attacker(attack, test_dataset, attack_args)

attack_results = attacker.attack_dataset()



  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 1/500 [01:07<9:20:20, 67.38s/it]

[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   0%|          | 1/500 [01:07<9:20:31, 67.40s/it]

[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   0%|          | 2/500 [01:18<5:25:36, 39.23s/it]

[Succeeded / Failed / Skipped / Total] 0 / 2 / 0 / 2:   0%|          | 2/500 [01:18<5:25:40, 39.24s/it]

[Succeeded / Failed / Skipped / Total] 0 / 2 / 0 / 2:   1%|          | 3/500 [01:23<3:50:37, 27.84s/it]

[Succeeded / Failed / Skipped / Total] 1 / 2 / 0 / 3:   1%|          | 3/500 [01:23<3:50:40, 27.85s/it]

[Succeeded / Failed / Skipped / Total] 1 / 2 / 0 / 3:   1%|          | 4/500 [01:30<3:07:33, 22.69s/it]

[Succeeded / Failed / Skipped / Total] 2 / 2 / 0 / 4:   1%|          | 4/500 [01:30<3:07:37, 22.70s/it]

[Succeeded / Failed / Skipped / Total] 2 / 2 / 0 / 4:   1%|          | 5/500 [01:33<2:34:04, 18.68s/it]

[Succeeded / Failed / Skipped / Total] 3 / 2 / 0 / 5:   1%|       



[Succeeded / Failed / Skipped / Total] 66 / 28 / 6 / 100:  20%|██        | 101/500 [26:21<1:44:06, 15.66s/it]

[Succeeded / Failed / Skipped / Total] 66 / 29 / 6 / 101:  20%|██        | 101/500 [26:21<1:44:06, 15.66s/it]

[Succeeded / Failed / Skipped / Total] 66 / 29 / 6 / 101:  20%|██        | 102/500 [26:24<1:43:03, 15.54s/it]

[Succeeded / Failed / Skipped / Total] 67 / 29 / 6 / 102:  20%|██        | 102/500 [26:24<1:43:03, 15.54s/it]

[Succeeded / Failed / Skipped / Total] 67 / 29 / 6 / 102:  21%|██        | 103/500 [27:27<1:45:49, 15.99s/it]

[Succeeded / Failed / Skipped / Total] 67 / 30 / 6 / 103:  21%|██        | 103/500 [27:27<1:45:49, 15.99s/it]

[Succeeded / Failed / Skipped / Total] 67 / 30 / 6 / 103:  21%|██        | 104/500 [27:40<1:45:24, 15.97s/it]

[Succeeded / Failed / Skipped / Total] 68 / 30 / 6 / 104:  21%|██        | 104/500 [27:40<1:45:24, 15.97s/it]

[Succeeded / Failed / Skipped / Total] 68 / 30 / 6 / 104:  21%|██        | 105/500 [27:53<1:44:56, 15.94s/it]




[Succeeded / Failed / Skipped / Total] 132 / 59 / 9 / 200:  40%|████      | 201/500 [55:41<1:22:51, 16.63s/it]

[Succeeded / Failed / Skipped / Total] 132 / 60 / 9 / 201:  40%|████      | 201/500 [55:41<1:22:51, 16.63s/it]

[Succeeded / Failed / Skipped / Total] 132 / 60 / 9 / 201:  40%|████      | 202/500 [55:42<1:22:11, 16.55s/it]

[Succeeded / Failed / Skipped / Total] 133 / 60 / 9 / 202:  40%|████      | 202/500 [55:42<1:22:11, 16.55s/it]

[Succeeded / Failed / Skipped / Total] 133 / 60 / 9 / 202:  41%|████      | 203/500 [56:10<1:22:11, 16.60s/it]

[Succeeded / Failed / Skipped / Total] 133 / 61 / 9 / 203:  41%|████      | 203/500 [56:10<1:22:11, 16.60s/it]

[Succeeded / Failed / Skipped / Total] 133 / 61 / 9 / 203:  41%|████      | 204/500 [56:37<1:22:09, 16.65s/it]

[Succeeded / Failed / Skipped / Total] 133 / 62 / 9 / 204:  41%|████      | 204/500 [56:37<1:22:09, 16.65s/it]

[Succeeded / Failed / Skipped / Total] 133 / 62 / 9 / 204:  41%|████      | 205/500 [57:04<1:22:07, 16



[Succeeded / Failed / Skipped / Total] 187 / 99 / 14 / 300:  60%|██████    | 301/500 [1:25:12<56:20, 16.98s/it]

[Succeeded / Failed / Skipped / Total] 187 / 100 / 14 / 301:  60%|██████    | 301/500 [1:25:12<56:20, 16.99s/it]

[Succeeded / Failed / Skipped / Total] 187 / 100 / 14 / 301:  60%|██████    | 302/500 [1:25:30<56:03, 16.99s/it]

[Succeeded / Failed / Skipped / Total] 187 / 101 / 14 / 302:  60%|██████    | 302/500 [1:25:30<56:03, 16.99s/it]

[Succeeded / Failed / Skipped / Total] 187 / 101 / 14 / 302:  61%|██████    | 303/500 [1:25:45<55:45, 16.98s/it]

[Succeeded / Failed / Skipped / Total] 188 / 101 / 14 / 303:  61%|██████    | 303/500 [1:25:45<55:45, 16.98s/it]

[Succeeded / Failed / Skipped / Total] 188 / 101 / 14 / 303:  61%|██████    | 304/500 [1:28:16<56:54, 17.42s/it]

[Succeeded / Failed / Skipped / Total] 188 / 102 / 14 / 304:  61%|██████    | 304/500 [1:28:16<56:54, 17.42s/it]

[Succeeded / Failed / Skipped / Total] 188 / 102 / 14 / 304:  61%|██████    | 305/500 [



[Succeeded / Failed / Skipped / Total] 252 / 127 / 21 / 400:  80%|████████  | 401/500 [1:58:34<29:16, 17.74s/it]

[Succeeded / Failed / Skipped / Total] 253 / 127 / 21 / 401:  80%|████████  | 401/500 [1:58:34<29:16, 17.74s/it]

[Succeeded / Failed / Skipped / Total] 253 / 127 / 21 / 401:  80%|████████  | 402/500 [1:58:36<28:54, 17.70s/it]

[Succeeded / Failed / Skipped / Total] 254 / 127 / 21 / 402:  80%|████████  | 402/500 [1:58:36<28:54, 17.70s/it]

[Succeeded / Failed / Skipped / Total] 254 / 127 / 21 / 402:  81%|████████  | 403/500 [1:58:55<28:37, 17.71s/it]

[Succeeded / Failed / Skipped / Total] 254 / 128 / 21 / 403:  81%|████████  | 403/500 [1:58:55<28:37, 17.71s/it]

[Succeeded / Failed / Skipped / Total] 254 / 128 / 21 / 403:  81%|████████  | 404/500 [2:00:28<28:37, 17.89s/it]

[Succeeded / Failed / Skipped / Total] 254 / 129 / 21 / 404:  81%|████████  | 404/500 [2:00:28<28:37, 17.89s/it]

[Succeeded / Failed / Skipped / Total] 254 / 129 / 21 / 404:  81%|████████  | 405/500 

...

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (99831 > 1024). Running this sequence through the model will result in indexing errors
